In [9]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [10]:
# Import from the crewAI libray.
from crewai import Agent, Task, Crew

# from utils import get_openai_api_key, pretty_print_result
# from utils import get_serper_api_key

# import library
from dotenv import load_dotenv
import os #needed to call function
import openai

In [13]:
# setup configuration function
def configure():
    load_dotenv()
    
# load configuration
configure()

# get the key
openai_api_key = os.getenv("OPENAI_API_KEY")
serper_api_key = os.getenv("SERPER_API_KEY")

os.environ["OPENAI_MODEL_NAME"] = 'gpt-4-turbo'


## Creating Agents

In [14]:
sales_rep_agent = Agent(
    role="Sales Representative",
    goal="Identify high-value leads that match "
         "our ideal customer profile",
    backstory=(
        "As a part of the dynamic sales team at CrewAI, "
        "your mission is to scour "
        "the digital landscape for potential leads. "
        "Armed with cutting-edge tools "
        "and a strategic mindset, you analyze data, "
        "trends, and interactions to "
        "unearth opportunities that others might overlook. "
        "Your work is crucial in paving the way "
        "for meaningful engagements and driving the company's growth."
    ),
    allow_delegation=False,
    verbose=True
)

In [15]:
lead_sales_rep_agent = Agent(
    role="Lead Sales Representative",
    goal="Nurture leads with personalized, compelling communications",
    backstory=(
        "Within the vibrant ecosystem of CrewAI's sales department, "
        "you stand out as the bridge between potential clients "
        "and the solutions they need."
        "By creating engaging, personalized messages, "
        "you not only inform leads about our offerings "
        "but also make them feel seen and heard."
        "Your role is pivotal in converting interest "
        "into action, guiding leads through the journey "
        "from curiosity to commitment."
    ),
    allow_delegation=False,
    verbose=True
)

## Creating Tools

### crewAI Tools

In [27]:
from crewai_tools import DirectoryReadTool, \
                         FileReadTool, \
                         SerperDevTool

In [28]:
# read directory 
directory_read_tool = DirectoryReadTool(directory='./instructions') # 
# read file
file_read_tool = FileReadTool()
# search the web
search_tool = SerperDevTool()

## Creating Tools

### crewAI Tools

In [29]:
from crewai_tools import DirectoryReadTool, \
                         FileReadTool, \
                         SerperDevTool

In [30]:
directory_read_tool = DirectoryReadTool(directory='./instructions')
file_read_tool = FileReadTool()
search_tool = SerperDevTool()

### Custom Tool
- Create a custom tool using crewAi's [BaseTool](https://docs.crewai.com/core-concepts/Tools/#subclassing-basetool) class

In [ ]:
# 
from crewai_tools import BaseTool

- Every Tool needs to have a `name` and a `description`.
- For simplicity and classroom purposes, `SentimentAnalysisTool` will return `positive` for every text.
- When running locally, you can customize the code with your logic in the `_run` function.

In [22]:
class SentimentAnalysisTool(BaseTool):
    name: str ="Sentiment Analysis Tool"
    description: str = ("Analyzes the sentiment of text "
         "to ensure positive and engaging communication.")
    
    def _run(self, text: str) -> str:
        # Your custom code tool goes here
        return "positive"

In [23]:
sentiment_analysis_tool = SentimentAnalysisTool()

## Creating Tasks

- The Lead Profiling Task is using crewAI Tools.

In [31]:
lead_profiling_task = Task(
    description=(
        "Conduct an in-depth analysis of {lead_name}, "
        "a company in the {industry} sector "
        "that recently showed interest in our solutions. "
        "Utilize all available data sources "
        "to compile a detailed profile, "
        "focusing on key decision-makers, recent business "
        "developments, and potential needs "
        "that align with our offerings. "
        "This task is crucial for tailoring "
        "our engagement strategy effectively.\n"
        "Don't make assumptions and "
        "only use information you absolutely sure about."
    ),
    expected_output=(
        "A comprehensive report on {lead_name}, "
        "including company background, "
        "key personnel, recent milestones, and identified needs. "
        "Highlight potential areas where "
        "our solutions can provide value, "
        "and suggest personalized engagement strategies."
    ),
    tools=[directory_read_tool, file_read_tool, search_tool],
    agent=sales_rep_agent,
)

- The Personalized Outreach Task is using your custom Tool `SentimentAnalysisTool`, as well as crewAI's `SerperDevTool` (search_tool).

In [32]:
personalized_outreach_task = Task(
    description=(
        "Using the insights gathered from "
        "the lead profiling report on {lead_name}, "
        "craft a personalized outreach campaign "
        "aimed at {key_decision_maker}, "
        "the {position} of {lead_name}. "
        "The campaign should address their recent {milestone} "
        "and how our solutions can support their goals. "
        "Your communication must resonate "
        "with {lead_name}'s company culture and values, "
        "demonstrating a deep understanding of "
        "their business and needs.\n"
        "Don't make assumptions and only "
        "use information you absolutely sure about."
    ),
    expected_output=(
        "A series of personalized email drafts "
        "tailored to {lead_name}, "
        "specifically targeting {key_decision_maker}."
        "Each draft should include "
        "a compelling narrative that connects our solutions "
        "with their recent achievements and future goals. "
        "Ensure the tone is engaging, professional, "
        "and aligned with {lead_name}'s corporate identity."
    ),
    tools=[sentiment_analysis_tool, search_tool],
    agent=lead_sales_rep_agent,
)

## Creating the Crew

In [33]:
crew = Crew(
    agents=[sales_rep_agent, 
            lead_sales_rep_agent],
    
    tasks=[lead_profiling_task, 
           personalized_outreach_task],
	
    verbose=2,
	memory=True
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [34]:
inputs = {
    "lead_name": "DeepLearningAI",
    "industry": "Online Learning Platform",
    "key_decision_maker": "Andrew Ng",
    "position": "CEO",
    "milestone": "product launch"
}

result = crew.kickoff(inputs=inputs)

 [DEBUG]: == Working Agent: Sales Representative
 [INFO]: == Starting Task: Conduct an in-depth analysis of DeepLearningAI, a company in the Online Learning Platform sector that recently showed interest in our solutions. Utilize all available data sources to compile a detailed profile, focusing on key decision-makers, recent business developments, and potential needs that align with our offerings. This task is crucial for tailoring our engagement strategy effectively.
Don't make assumptions and only use information you absolutely sure about.


> Entering new CrewAgentExecutor chain...
Thought: To start gathering comprehensive information on DeepLearningAI, I need to analyze existing data about the company. I will begin by searching the internet to find general information on DeepLearningAI, including its background, key personnel, and recent business developments.

Action: Search the internet
Action Input: {"search_query": "DeepLearningAI company profile"} 


Search results: Title: Abo

In [35]:
# Display the final result as Markdown.
from IPython.display import Markdown
Markdown(result)

1. **Email regarding Educational Platform Enhancement**:
   Subject: Empowering DeepLearningAI's Mission Through Enhanced Educational Tools
   ```
   Dear Andrew,

   I hope this message finds you well. I'm truly inspired by DeepLearningAI's commitment to making AI education accessible globally. Your recent initiatives to expand your learning platform are commendable.

   At CrewAI, we understand the challenges of scaling educational content while maintaining high engagement and reliability. Our Flows system can seamlessly integrate with your platform to automate and streamline processes, ensuring a consistent and enriching learning experience for users worldwide.

   I would be thrilled to explore how we can collaborate to enhance your educational offerings. Perhaps we could schedule a call to discuss this further?

   Warm regards,
   [Your Name]
   ```

2. **Email on Community Engagement**:
   Subject: Strengthening Global AI Community Connections with CrewAI
   ```
   Dear Andrew,

   The vibrant community engagement at DeepLearningAI is a testament to your leadership and vision. The dedicated forums and global connections you're fostering are essential for the future of AI education.

   CrewAI's Flows can provide advanced automation solutions that enhance user interactions and operational efficiency within your community forums. This could further enrich the community's experience and engagement, driving more vibrant discussions and learning opportunities.

   I'd love to discuss how our technology could support your community initiatives. Could we set up a time to talk next week?

   Best,
   [Your Name]
   ```

3. **Email about AI Developer Conference Support**:
   Subject: Boosting DeepLearningAI’s Developer Conferences with Advanced AI Solutions
   ```
   Dear Andrew,

   Your AI Developer Conferences are a cornerstone for practical AI learning. At CrewAI, we are excited about the potential for our technologies to add value to these events.

   Our Flows system can introduce new levels of interaction and learning experiences at your conferences, from live demos of AI workflows to hands-on sessions that utilize our automation tools in real-time.

   It would be an honor to discuss how we can be a part of your next event, perhaps providing a workshop or a tech demo that aligns with your conference themes.

   Looking forward to your thoughts,

   Sincerely,
   [Your Name]
   ```

These emails are crafted to specifically address key aspects of DeepLearningAI's operations and how CrewAI's solutions can be aligned to support and enhance their goals and initiatives. Each email is personalized, reflecting an understanding of their recent developments and long-term mission.